# Langchain

### Prompts

- Get OpenAI key

In [1]:
! pip install -r ../../requirements.txt

In [2]:
import dotenv

# OPENAI_API_KEY-
dotenv.load_dotenv('.env')

True

In [3]:
from langchain_core.prompts import ChatPromptTemplate

# 'human', 'user', 'ai', 'assistant', or 'system'
prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI bot. Your name is {name}."),
            ("human", "Hello, how are you doing?"),
            ("ai", "I'm doing well, thanks!"),
            ("human", "{user_input}"),
        ])

final_prompt = prompt.invoke({
        "name": "Bob",
        "user_input": "What is your name?"
    })
final_prompt

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'), HumanMessage(content='Hello, how are you doing?'), AIMessage(content="I'm doing well, thanks!"), HumanMessage(content='What is your name?')])

In [4]:
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot."),
    MessagesPlaceholder(variable_name="conversation", optional=True)
])

# Conversation can be retireved from other sources like Vector store, other APIs
final_prompt = prompt.invoke(
    {
        "conversation": [
            ("human", "Hi!"),
            ("ai", "How can I assist you today?"),
            ("human", "Can you make me an ice cream sundae?"),
            ("ai", "No.")
        ]
    }
)

final_prompt

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot.'), HumanMessage(content='Hi!'), AIMessage(content='How can I assist you today?'), HumanMessage(content='Can you make me an ice cream sundae?'), AIMessage(content='No.')])

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm.invoke("Hello, how are you doing?")

AIMessage(content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 14, 'total_tokens': 43}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_9b0abffe81', 'finish_reason': 'stop', 'logprobs': None}, id='run-7c3d8bc6-7bdf-404f-8a29-848478c315f8-0')

In [6]:
# different model
llm = ChatOpenAI(model="gpt-4o")
llm.invoke("Hello, how are you doing?")

AIMessage(content="Hello! I'm an AI, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 14, 'total_tokens': 42}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_3cd8b62c3b', 'finish_reason': 'stop', 'logprobs': None}, id='run-b44ccc34-e4a4-4b6d-ac2d-f5bb6e39605c-0')

In [8]:
# different Hyperparameters
print(ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=100).invoke("Hello, how are you doing?"))
print(ChatOpenAI(model="gpt-4o", temperature=2, max_tokens=100).invoke("Hello, how are you doing?"))

content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?" response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 14, 'total_tokens': 44}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_c832e4513b', 'finish_reason': 'stop', 'logprobs': None} id='run-3b565ce9-9304-4ce3-9860-9747577a780e-0'
content="Hello! I'm an AI, so I don't have feelings, but I'm ready to help with whatever you need. How can I assist you today?" response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 14, 'total_tokens': 43}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'stop', 'logprobs': None} id='run-8a928310-f3a1-4026-ac1c-2c03ca18fec0-0'


In [9]:
### Different Output Parser
from langchain_core.output_parsers import StrOutputParser
chain = llm | StrOutputParser()
chain.invoke("Hello, how are you doing?")

"Hello! I'm an AI, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?"

# Chains

In [10]:
prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI bot. Your name is {name}."),
            ("human", "Hello, how are you doing?"),
            ("ai", "I'm doing well, thanks!"),
            ("human", "{user_input}"),
        ])
llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=100)
parser = StrOutputParser()

chain = prompt | llm | parser
chain.invoke({
        "name": "Bob",
        "user_input": "What is your name?"
})

'My name is Bob. How can I assist you today?'

In [12]:
## Streaming invocation
stream = chain.stream({
        "name": "Bob",
        "user_input": "What is your name?"
})


for token in stream:
    print(token)


My
 name
 is
 Bob
.
 How
 can
 I
 assist
 you
 today
?



# saving and loading chain

In [13]:
from langchain_core.load import dumpd
import json

json_str_chain = dumpd(chain)
with open("./chain.json", "w") as fp:
    json.dump(json_str_chain, fp, indent=4)

In [14]:
chain.middle[0].lc_secrets

{'openai_api_key': 'OPENAI_API_KEY'}

In [15]:
from langchain_core.load import load
import os

with open("./chain.json", "r") as fp:
    chain = load(json.load(fp), secrets_map={"OPENAI_API_KEY": os.getenv("OPENAI_API_KEY")})
chain

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `load` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


ChatPromptTemplate(input_variables=['name', 'user_input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], template='You are a helpful AI bot. Your name is {name}.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Hello, how are you doing?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="I'm doing well, thanks!")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f5a02d32920>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f5a02d50250>, model_name='gpt-4o', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=100)
| StrOutputParser()

In [16]:
chain.invoke({
        "name": "Bob",
        "user_input": "What is your name?"
})

'My name is Bob. How can I assist you today?'